In [1]:
# Load packages
import ee
import pandas as pd
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import *
import geopandas as gpd
import numpy as np
import datatable as dt
from datatable import f
import time
from pprint import pprint
import os, re
from gee_subset import gee_subset
import shapely.geometry
import re
import geemap
from geextract import ts_extract

Script to extract Landsat 8 and GPM precipitation time series for all restoration plots using GEE

In [2]:
# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [3]:
# Select country
country = 'Ethiopia'

In [4]:
# Import the plot geojson with the centroids
plots = gpd.read_file("output/plot_data/" + country + "/" + country + "_plot_centroid.GeoJSON")
plots_dt = dt.fread("output/plot_data/" + country + "/" + country + "_plot_centroid.csv")

In [4]:
# Import validation points (for validation process)
plots = gpd.read_file("data/validation_points.GeoJSON")
plots_dt = dt.fread("data/validation_points.csv")

In [5]:
# Create columns with x and y coordinates
plots['lon'] = plots['geometry'].x
plots['lat'] = plots['geometry'].y 

In [6]:
x_dt = dt.Frame(plots['lon'])
y_dt = dt.Frame(plots['lat'])
plots_dt.cbind(x_dt, y_dt)

In [8]:
# Import the plot geosjon with the polygon geometries
plotsPol = gpd.read_file("output/plot_data/" + country + "/" + country + "_plots_all.GeoJSON")
plotsPol_dt = dt.fread("output/plot_data/" + country + "/" + country + "_plots_all.csv")

In [7]:
# Create ee.Multipoint (for GPM extraction)
geom_list = []
for i in range(x_dt.nrows):
  point = [x_dt[i, 'lon'], y_dt[i, 'lat']]
  geom_list.append(point)

centroid_multi = ee.Geometry.MultiPoint(geom_list)

In [8]:
# Time series settings
start_date = datetime(2013, 1, 1)
end_date = datetime(2023, 1, 1)
#end_date = datetime.today()

**Extract L8 polygons**

In [9]:
# Select bands and resolution to extract from landsat 8
bands = ["SR_B4", "SR_B5", "QA_PIXEL"]
#bands = ["SR_B3", "SR_B4", "SR_B5", "QA_PIXEL"]
scale = 30
product='LANDSAT/LC08/C02/T1_L2'
#roduct='LANDSAT/LE07/C02/T1_L2'

In [10]:
# Create function to get yearly dates for the yearly extraction
def get_year_dates(start, end):
    yearly_dates = [start]
    year_diff = (end.year - start.year) + 1

    for i in range(1, year_diff+1):
        next_year = start_date + relativedelta(years =+ i) 
        yearly_dates.append(next_year)

    return yearly_dates

year_dates = get_year_dates(start_date, end_date)

In [ ]:
# Extract L8 for restoration sites
refl_l8_pol = dt.Frame()
for pl in range(len(plotsPol)):
    print(pl)

    g = [i for i in plotsPol.geometry]
    x,y = g[pl].exterior.coords.xy
    cords = np.dstack((x,y)).tolist()
    geometry = ee.Geometry.Polygon(cords)
    geometry = geometry.buffer(0.000063)

    # Check if geometry exceeds critical size (15ha)
    if plotsPol['Hectare'].iloc[pl] > 50:
        print('Area too large --> extracting per year')
        # Extract data per year to prevent GEE size error
        for yr in range(len(year_dates)-1):
            try:
                col = ee.ImageCollection(product).\
                    select(tuple(bands)).\
                    filterDate(year_dates[yr], year_dates[yr+1]).filterBounds(geometry)


                # Make a df
                region = col.getRegion(geometry, int(scale)).getInfo()
                df = pd.DataFrame.from_records(region[1:len(region)])
                df.columns = region[0]
                df = df[['time', 'SR_B4', 'SR_B5', 'QA_PIXEL']]
  
                df.time = df.time / 1000
                df['time'] = pd.to_datetime(df['time'], unit = 's')
                df.rename(columns = {'time': 'date'}, inplace = True)
                df.sort_values(by = 'date')

                # Transform to dt
                l8_out = dt.Frame(df)

                # Create column with plotID 
                l8_out['plotID'] = plotsPol['plotID'].iloc[pl]
                refl_l8_pol.rbind(l8_out)

            except:
                pass
                
    else:
        col = ee.ImageCollection(product).\
            select(tuple(bands)).\
            filterDate(start_date, end_date).filterBounds(geometry)

        region = col.getRegion(geometry, int(scale)).getInfo()

        # If no pixels in geometry, take centroid of plot
        if len(region) == 1:
            print('Not enough pixels in geometry (taking centroid)')
            
            geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])
            col = ee.ImageCollection(product).\
                select(tuple(bands)).\
                filterDate(start_date, end_date).filterBounds(geometry)
      
        region = col.getRegion(geometry, int(scale)).getInfo()
        df = pd.DataFrame.from_records(region[1:len(region)])
        df.columns = region[0]
        df = df[['time', 'SR_B4', 'SR_B5', 'QA_PIXEL']]
  
        df.time = df.time / 1000
        df['time'] = pd.to_datetime(df['time'], unit = 's')
        df.rename(columns = {'time': 'date'}, inplace = True)
        df.sort_values(by = 'date')

        # Transform to dt
        l8_out = dt.Frame(df)

        # Create column with plotID 
        l8_out['plotID'] = plotsPol['plotID'].iloc[pl]

        refl_l8_pol.rbind(l8_out)

In [11]:
# Extract L8 for validation points
refl_l8_val = dt.Frame()
for pl in range(len(plots)):
    print(pl)

    # Create point geometry and extract
    geometry = ee.Geometry.Point([plots_dt[pl,'lon'], plots_dt[pl, 'lat']])
    col = ee.ImageCollection(product).\
        select(tuple(bands)).\
        filterDate(start_date, end_date).filterBounds(geometry)

    # Convert to df
    region = col.getRegion(geometry, int(scale)).getInfo()
    df = pd.DataFrame.from_records(region[1:len(region)])
    df.columns = region[0]
    df = df[['time', 'SR_B4', 'SR_B5', 'QA_PIXEL']]
  
    df.time = df.time / 1000
    df['time'] = pd.to_datetime(df['time'], unit = 's')
    df.rename(columns = {'time': 'date'}, inplace = True)
    df.sort_values(by = 'date')

    # Transform to dt
    l8_out = dt.Frame(df)

    # Create column with plotID 
    l8_out['plotID'] = plots['plotID'].iloc[pl]
    
    refl_l8_val.rbind(l8_out)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [146]:
# Write the country reflectances
refl_l8_pol.to_csv("output/gee/" + country + "_refl_l8_pol.csv")

In [12]:
# Write the validation point reflectances
refl_l8_val.to_csv("output/gee/Val_refl_l8.csv")

In [53]:
# Read reflectance file
refl_l8_pol = dt.fread("output/gee/Ethiopia_refl_l8_pol.csv")

**Extract Precipitation data (GPM)**

In [13]:
## Set parameters for the time series
start_ts = datetime(2012, 9, 1) # 2 months added to ts period cause of lag (t-2 is also included in the model)
end_ts = datetime(2023, 4, 1) 

# Specify number of days in period of interest
d0 = datetime(start_ts.year, start_ts.month, start_ts.day)
d1 = datetime(end_ts.year, end_ts.month, 1)
delta = d1 - d0
days = delta.days

# number of months in period
def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month
months_ts = diff_month(d1, d0)

# Create list with the dates off all the observations
months_date = []
for m in range(months_ts):
  first_month = start_ts
  next_month = first_month + relativedelta(months =+ m)
  months_date.append(next_month)

In [14]:
# Extract all the precipitation data for all sites
gpm = ee.ImageCollection('NASA/GPM_L3/IMERG_V06').\
       select('precipitationCal').\
       filterDate(start_ts, end_ts).filterBounds(centroid_multi)

In [15]:
# Create a function to go over the FeatureCollection and take the monthly sum
def GPMsum(img_collection):
  mylist = ee.List([])
  for m in range(months_ts):

    ini = start_ts + relativedelta(months=+m)
    end = ini + relativedelta(months=+1) + relativedelta(days=-1)

    sum_image = img_collection.filterDate(ini,end).select(0).sum()
    mylist = mylist.add(sum_image.set('system:time_start', ini))
  return ee.ImageCollection.fromImages(mylist)

In [16]:
# Apply the 'GPMsum' function to create FeatureCollection with monthly sum 
monthlyGPM = ee.ImageCollection(GPMsum(gpm))
# Sort FeatureCollection by date and create single image with bands
monthlyGPM_stack = monthlyGPM.sort('system:time_start').toBands().multiply(0.5)

In [17]:
img_todrive = {
    'description': country + '_GPM_stack',
    'folder': 'Regreening_Africa',
    'scale': 11000,
    'maxPixels': 1e13,
    'region': centroid_multi,
    'fileFormat': 'GeoTIFF'}

task = ee.batch.Export.image.toDrive(monthlyGPM_stack, **img_todrive)
task.start()

In [19]:
# Check the status of the upload task
task.status()
ee.data.listOperations()

[{'name': 'projects/earthengine-legacy/operations/6VOHTM6SM4R3TS2GY7MFXUOH',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'RUNNING',
   'description': 'Ethiopia_GPM_stack',
   'createTime': '2024-01-18T08:07:08.976782Z',
   'updateTime': '2024-01-18T08:10:13.068084Z',
   'startTime': '2024-01-18T08:07:11.920032Z',
   'type': 'EXPORT_IMAGE',
   'attempt': 1,
   'progress': 0.5454545454545454,
   'stages': [{'displayName': 'Create Local Files',
     'completeWorkUnits': 6,
     'totalWorkUnits': '10',
     'description': 'Computation and writing of temporary files.'},
    {'displayName': 'Write Files to Destination',
     'totalWorkUnits': '1',
     'description': 'Uploading of files to the export destination.'}]}},
 {'name': 'projects/earthengine-legacy/operations/7U5LWKUKUBG2GXD4NGJMKHSG',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'SUCCEEDED',
   'description': 'MODI

In [18]:
country

'Ethiopia'